In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')
#добавляю для себя, так у меня работают пакеты!

In [2]:
import pandas as pd
from tqdm import tqdm
import os
import re
from nltk import trigrams
from collections import Counter
import pymorphy2 # Лемматизация

## Скачиваем тексты wiki ##

In [20]:
import wikipedia

In [21]:
wikipedia.set_lang("ru")

In [28]:
random_pages_names = wikipedia.random(pages=500)

In [4]:
path_train_texts = 'C:/Users/Анна/YandexDisk/spell-check/wikiru/'

In [29]:
for random_name in tqdm(random_pages_names):
    page_text = []
    random_name = wikipedia.search(random_name)
    try:
        saved_page = wikipedia.page(random_name[0])
    except (wikipedia.exceptions.DisambiguationError,
            wikipedia.exceptions.PageError) as e:
        print("Error: {0}".format(e))
        continue
    try:
        page_text.append(saved_page.title)
        page_text.append(saved_page.summary)
        page_text.append(saved_page.content)
    except wikipedia.exceptions.DisambiguationError as e:
        print("Error: {0}".format(e))
        continue
             
   
    page_text = ' '.join(page_text)

    file_name = (path_train_texts+(saved_page.title) + '.txt')
    try:
        with open(file_name, 'w', encoding = 'utf-8') as fl:
            fl.write(page_text)
    except FileNotFoundError:
        continue

  3%|█▍                                       | 17/500 [00:57<27:06,  3.37s/it]C:\Program Files\Anaconda3\envs\mllecture\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Program Files\Anaconda3\envs\mllecture\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error: "Ново-Никольское" may refer to: 
Ново-Никольское
Ново-Никольское
Ново-Никольское
Ново-Никольское
Ново-Никольское
Ново-Никольское
Ново-Никольское
Новоникольское
Никольское


  5%|██▏                                      | 26/500 [01:26<26:24,  3.34s/it]

Error: "Насас" may refer to: 
Насас (муниципалитет)
Насас (Дуранго)
Насас (река)


  9%|███▌                                     | 43/500 [02:25<25:45,  3.38s/it]

...

Error: "Хамина (значения)" may refer to: 
греч.
Ха́мина
Лахдар-Хамина, Мохаммед
↑


 72%|████████████████████████████▉           | 361/500 [18:56<07:17,  3.15s/it]

Error: "Верхнеозерский" may refer to: 
Верхнеозерский
Верхнеозерский
Верхнее озеро


 75%|█████████████████████████████▊          | 373/500 [19:33<06:39,  3.15s/it]

ConnectionError: HTTPConnectionPool(host='ru.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?action=query&limit=10&list=search&srprop=&srsearch=%D0%9D%D0%B0%D0%B3%D0%B0%D1%81%D0%B0%D0%BA%D0%B8+%D0%90%D1%82%D0%BB%D0%B5%D1%82%D0%B8%D0%BA+%28%D1%81%D1%82%D0%B0%D0%B4%D0%B8%D0%BE%D0%BD%29&format=json&srlimit=10 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000000000F296908>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

In [6]:
def slurp(path):
    with open(path, 'r', encoding = 'utf-8') as file_object:
        return file_object.read()

## Препроцессинг ##

In [79]:
def preprocess(texts):
    preprocessed_texts_list = []
    for text in texts:
        text = re.sub(r'[A-Za-z]', '', text)
        text = re.sub(r'[А-Я]{2,}', '', text)#убираем аббревиатуры
        lower_text = text.lower()#приводим к нижнему регистру, заменяем тире, чтобы потом оставить дефисы в словах
        #lower_text = re.sub(r'(\d)+-(\d)+','', lower_text)#удаление цифр типа 1999-2001
        del_new_line = re.sub(r'\n', '', lower_text)
        no_ref_text = re.sub(r'==литература(.)+|==источники(.)+', '', del_new_line)#удаляем все, что после рубрики "Сссылки"
        no_num_text = re.sub(r'\d', '', no_ref_text)#удаляем цифры
        extracted_text = re.findall(r'[а-яё]+', no_num_text)
        #разделяем на токены - последовательности букв с дефисами
        extracted_text = ' '.join(extracted_text)
        preprocessed_texts_list.append(extracted_text)
    return preprocessed_texts_list

In [80]:
train_texts_list = []
for root, dirs, files in os.walk(path_train_texts):
    for file_name in tqdm(files):
        input_path = path_train_texts + file_name
        text = slurp(input_path)
        train_texts_list.append(text)
preprocessed_train_texts = preprocess(train_texts_list)    

100%|█████████████████████████████████████| 3538/3538 [00:04<00:00, 759.02it/s]


In [81]:
preprocessed_train_texts[14]

'число одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя свойства цифры первых пяти неотрицательных степеней числа являются элементами треугольника паскаля эндекахорон англ абстрактный правильный четырёхмерный политоп ячеек которого представляют собой полуикосаэдры число бога кубика рубика в метрике равно геометрия существует развёрток куба образующих подмножество гексамино и развёрток октаэдра образующих подмножество октиамондов из гексамино шестиклеточных полимино ровно являются чётными то есть при шахматной раскраске содержат чётное число светлых и чётное число тёмных квадратов дэвид а кларнер обнаружил 

In [82]:
train_wiki = pd.DataFrame(data = preprocessed_train_texts, columns=['wiki_texts'])

In [83]:
train_wiki.to_csv('train_wiki_2.csv', index = False, encoding = 'utf-8')

## Обучающий корпус - тексты Википедии ##

In [ ]:
train_wiki = pd.read_csv('/home/nst/mount/data/share/yd/comp_ling/spell-check/train_wiki2.csv')

In [ ]:
train_wiki = pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_wiki2.csv')

In [84]:
train_wiki.head()

,wiki_texts
0,долби долби лат астероид относящийся к группе ...
1,адрия адрия лат довольно крупный астероид из г...
2,гиппас гиппас лат др греч типичный троянский а...
3,гоголь гоголь лат типичный астероид главного п...
4,капрера капрера итал астероид главного пояса п...


## Обучающая выборка с ошибками

In [ ]:
with open('training.txt', encoding = 'utf-8') as f:
    train = f.read()

In [ ]:
train = train.split('\n\n')

In [ ]:
train_df = pd.DataFrame(data = train, columns = ['train'])

In [ ]:
train_df['wrong'], train_df['right'] = train_df['train'].str.split('\n',1).str

In [ ]:
train_df.pop('train');

In [ ]:
train_df.to_csv('train_df.csv', index = False, encoding="utf-8")

In [37]:
train_df =  pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_df.csv')

In [38]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,"Вальс в 5 па ( внимание, будет под другую музы...",Вальс в 5 па внимание будет под другую музыку ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


In [46]:
#уберем знаки препинания из обучающей выборки
train_df.wrong = [' '.join(re.findall(r'[а-я]+', text.lower())) for text in train_df.wrong]
train_df.right = [' '.join(re.findall(r'[а-я]+', text.lower())) for text in train_df.right]

In [47]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


## Импорт словарей ##

### rucorpora ###

In [ ]:
with open('/home/nst/mount/data/share/yd/comp_ling/spell-check/dict.opcorpora.txt') as fo:
    rucorpora_dict = fo.read()

In [43]:
with open('C:/Users/Анна/YandexDisk/spell-check/dict.opcorpora.txt', encoding = 'utf-8') as fo:
    rucorpora_dict = fo.read()

In [44]:
rucorpora_dict = re.findall(r'[А-ЯЁ]+', rucorpora_dict)
rucorpora_dict = ' '.join(rucorpora_dict)
rucorpora_dict = rucorpora_dict.lower()
#rucorpora_dict = rucorpora_dict.split()

In [45]:
rucorpora_dict[:20]

'ёж ежа ежу ежа ежом '

## Строим триграммы по тексту ##

In [48]:
def get_trigrams(text): #строим триграммы по тексту
    trigrams_list = []
    for word in text.split():
        trigram = [''.join(char) for char in list(trigrams(word))]
        if trigram:
            trigrams_list.append(' '.join(trigram))
        else:
            trigrams_list.append(word)   
   #tr = [' '.join([''.join(chars) for chars in list(trigrams(word))]) for word in text.split()]
    return trigrams_list

In [49]:
print(train_df.wrong[0])
print(get_trigrams(train_df.wrong[0]))
print(' '.join(get_trigrams(train_df.wrong[0])).split())

тоесть если дамой то только в том случае если потеряю работу в сентябре
['тое оес ест сть', 'есл сли', 'дам амо мой', 'то', 'тол оль льк ько', 'в', 'том', 'слу луч уча чае', 'есл сли', 'пот оте тер еря ряю', 'раб або бот оту', 'в', 'сен ент нтя тяб ябр бре']
['тое', 'оес', 'ест', 'сть', 'есл', 'сли', 'дам', 'амо', 'мой', 'то', 'тол', 'оль', 'льк', 'ько', 'в', 'том', 'слу', 'луч', 'уча', 'чае', 'есл', 'сли', 'пот', 'оте', 'тер', 'еря', 'ряю', 'раб', 'або', 'бот', 'оту', 'в', 'сен', 'ент', 'нтя', 'тяб', 'ябр', 'бре']


##  Вероятности триграмм в корпусе

In [50]:
def count_trigrams(corpus): #получаем вероятности триграмм в корпусе
    corpus_trigrams = []
    for text in corpus:
        tr = ' '.join(get_trigrams(text))
        corpus_trigrams.extend(tr.split())
    counts = dict(Counter(corpus_trigrams))
    for key, value in counts.items():
        counts[key] = value/len(corpus_trigrams)
    print(len(corpus_trigrams))    
    return counts

### Trigram wiki model

In [85]:
%%time
trigram_wiki_model = count_trigrams(train_wiki.wiki_texts)
#обучили 3-граммную модель на корпусе для получения вероятностей 3-грамм
#выводится общее число триграм в корпусе (c повторениями)

8297222
Wall time: 34.7 s


In [53]:
trigram_wiki_model.get('есл') #вероятность 3-граммы "есл"

0.0001625644903320455

In [54]:
trigram_wiki_model.get('зрх') #такой 3-граммы нет, модель не возвращает ничего

In [87]:
len(trigram_wiki_model)

13679

In [91]:
10/8297222 #какой порог взять для проверки слова по словарю?

1.2052226636818925e-06

In [88]:
trigram_wiki_model.items()

dict_items([('кцы', 1.2052226636818925e-07), ('ибч', 1.2052226636818925e-07), ('год', 0.0037295615327636164), ('люр', 1.6873117291546496e-06), ('физ', 0.00013606963872968568), ('охэ', 2.410445327363785e-07), ('язы', 0.0002588818281588705), ('юнк', 8.074991846668681e-06), ('жуб', 1.3257449300500819e-06), ('дып', 1.2052226636818925e-07), ('шуа', 2.1694007946274067e-06), ('лул', 5.182457453832138e-06), ('рее', 0.00012715099101843968), ('июм', 3.6156679910456776e-07), ('бэа', 1.2052226636818925e-07), ('до', 0.0006127352022158741), ('ктр', 0.0001806628772859157), ('тсы', 5.061935187463948e-06), ('ярк', 1.952460715164666e-05), ('мсб', 2.289923060995596e-06), ('ши', 2.0488785282592175e-06), ('тци', 3.6156679910456776e-07), ('мек', 3.36257123167248e-05), ('мяз', 1.2052226636818925e-07), ('аёж', 7.231335982091355e-07), ('омт', 8.436558645773248e-07), ('бни', 2.964847752657456e-05), ('тфу', 2.410445327363785e-07), ('сзы', 1.2052226636818925e-07), ('ьпт', 3.1697356054833774e-05), ('бне', 6.387680

## Поиск ошибок в текстах ##

In [94]:
def search_errors(text): #поиск ошибок в тексте
    errors = []
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    #print(trigram_list)
    for i, trigrams_string in enumerate(trigram_list):
        for trigram in trigrams_string.split():
            #print(trigram_model.get(tr))
            if not(trigram_wiki_model.get(trigram)) or (trigram_wiki_model.get(trigram) < (10/8297222)):
                #print(trigram)
                #print(text.split()[i])
                #if morph.parse(text.split()[i])[0].normal_form not in rucorpora_dict:
                if text.split()[i] not in rucorpora_dict:
                    errors.append(text.split()[i])
    return list(set(errors))

In [96]:
search_errors(train_df.wrong[10])

['своеи', 'самыи']

In [97]:
wiki_errors = []
for text in tqdm(train_df.wrong):
    wiki_errors.extend(search_errors(text))


 25%|█████████▊                             | 757/3000 [00:21<01:03, 35.12it/s]


 48%|██████████████████▏                   | 1434/3000 [00:41<00:45, 34.22it/s]


 73%|███████████████████████████▊          | 2195/3000 [01:03<00:23, 34.50it/s]


100%|██████████████████████████████████████| 3000/3000 [01:26<00:00, 34.68it/s]

In [98]:
len(wiki_errors)

556

In [101]:
print(wiki_errors[:100])

['такойже', 'недюженные', 'пейзажы', 'своеи', 'самыи', 'щастье', 'никоглда', 'отличаються', 'расжился', 'подощрительно', 'преподше', 'замерзнуть', 'можнго', 'првязанность', 'любю', 'йпри', 'дабаа', 'поведйте', 'воопще', 'нмного', 'черезчур', 'неопределнное', 'обчным', 'всмысле', 'отношуся', 'мущщину', 'мачовсе', 'божеупаси', 'вопщем', 'болезньматки', 'ндфл', 'теща', 'лучшым', 'кагбэ', 'рыпки', 'каааак', 'тогдя', 'совршенно', 'нащщот', 'дефачка', 'оооочень', 'оплаьа', 'мужык', 'кефирчиком', 'обяснил', 'проблемнная', 'внутрениий', 'элкетроинструменты', 'всеже', 'хочеца', 'блетчли', 'такскаать', 'воопщим', 'разделеннный', 'расщифровать', 'прощаите', 'реклма', 'очеь', 'ооочень', 'хваостаюсь', 'маленбких', 'саиые', 'дежурцве', 'хотябы', 'наконецто', 'каааак', 'оооочень', 'щастье', 'собссно', 'продкты', 'удеается', 'сногсшибальной', 'бплгодаря', 'блекнет', 'посдравляю', 'ооочень', 'сеъли', 'подргрузился', 'всвязи', 'прошедшых', 'всмысле', 'вооот', 'обязаательно', 'погкуляем', 'удалитъ', 'мор

## Генерация кандидатов

In [102]:
def get_all_candidates(word):
    letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    cursor = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    delete = [L + R[1:] for (L,R) in cursor if R]
    insert = [L + l + R for l in letters for (L,R) in cursor]
    replace = [L + l + R[1:] for l in letters for (L,R) in cursor if R]
    transpose = [L + R[1] + R[0] + R[2:] for (L,R) in cursor if len(R)>1]
    return set(delete + insert + replace + transpose)

In [103]:
def kill_candidates(error_word):
    candidates = []
    #kills = []
    for index, word in enumerate(get_all_candidates(error_word)):
        for trigram in ''.join(get_trigrams(word)).split():
            if not(trigram_wiki_model.get(trigram)):
                break
        if ' '+ word + ' ' in rucorpora_dict:
                    candidates.append(word)
    return candidates

In [104]:
print(len(get_all_candidates('пейзажы')))

495


In [105]:
%%time
print(kill_candidates('пейзажы'))

['пейзаж', 'пейзажа', 'пейзажны', 'пейзаны', 'пейзаже', 'пейзажу', 'пейзажи']
Wall time: 48 s


## Вероятности на словарных биграммах nltk

In [106]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [108]:
wiki_texts = train_wiki.wiki_texts
wiki_tokens = [word for text in wiki_texts for word in text.split()]
print(wiki_tokens[10:20])

['марса', 'он', 'был', 'открыт', 'декабря', 'года', 'американским', 'астрономом', 'любителем', 'чарльзом']


In [109]:
wiki_freq = nltk.FreqDist(wiki_tokens) #частотный словарь
#wiki_freq = Counter(wiki_tokens)
wiki_freq

FreqDist({'кирилуша': 1,
          'антииудейское': 1,
          'вьетнамцами': 2,
          'рекордом': 19,
          'роструп': 8,
          'зонтики': 3,
          'виргинскими': 1,
          'музыкальный': 94,
          'масштаба': 10,
          'засеки': 2,
          'осовремененной': 1,
          'станькову': 1,
          'перехватывает': 2,
          'цедрой': 1,
          'служанки': 7,
          'серебристая': 5,
          'моббингу': 2,
          'августинца': 1,
          'брэдфорд': 3,
          'доведя': 3,
          'преобразованные': 2,
          'перформансы': 1,
          'пристрастился': 2,
          'пырей': 2,
          'калифорния': 41,
          'монашества': 3,
          'перепады': 8,
          'изучил': 6,
          'сталинск': 1,
          'красоте': 4,
          'языкисогласно': 1,
          'артистический': 1,
          'каракалпакии': 1,
          'юнчен': 7,
          'экспертом': 6,
          'сяньби': 1,
          'телеграфами': 2,
          'абоненту': 

In [112]:
wiki_cfreq = nltk.ConditionalFreqDist(nltk.bigrams(wiki_tokens)) #словарь словарей для условных частот
#wiki_cfreq['и']

In [113]:
cprob = nltk.ConditionalProbDist(wiki_cfreq, nltk.MLEProbDist) #словарь условных вероятностей

print('p(русский язык) = %1.4f' %cprob['русский'].prob('язык'))
print('p(в доме) = %1.4f' %cprob['в'].prob('доме'))
print('p(солнечной системе) = %1.4f' %cprob['солнечной'].prob('системе'))

p(русский язык) = 0.1279
p(в доме) = 0.0011
p(солнечной системе) = 0.2558


In [156]:
def you_re_simply_the_best(candidates, left_context, right_context):
    s = sum(dict(wiki_freq).values())
    #cand_prob = dict(wiki_freq).get(cand)/s #априорная вероятность кандидата
    
    #считаем условные вероятности в контексте
    if left_context: #если есть слово слева от ошибочного
        if right_context: #если есть слово справа
            cand_prob = [(cand, cprob[left_context].prob(cand) * cprob[cand].prob(right_context)) for cand in candidates]
        else:
            cand_prob = [(cand, cprob[left_context].prob(cand)) for cand in candidates]
            
    else:
        cand_prob = [(cand, cprob[cand].prob(right_context)) for cand in candidates]
        
    cand_rate = sorted(cand_prob, key = lambda column:column[1], reverse = True) #сортируем по вероятности 
    
    if not(cand_rate[0][1]):
        cand_aprior = [(cand, dict(wiki_freq).get(cand)/s) if dict(wiki_freq).get(cand) else (cand, 0) for cand in candidates]
        cand_rate = sorted(cand_aprior, key = lambda column:column[1], reverse = True)
        
    return cand_rate[0][0]

In [117]:
for text in train_df.wrong:
    if 'пейзажы' in text:
        print(text)

пейзажы за окном какие то незнакомые


In [158]:
train_df[train_df.wrong == 'пейзажы за окном какие то незнакомые']

,wrong,right
9,пейзажы за окном какие то незнакомые,пейзажи за окном какие то незнакомые


In [151]:
dict(wiki_freq).get('пейзажи')

12

In [157]:
you_re_simply_the_best(['пейзаж', 'пейзажа', 'пейзажны', 'пейзаны', 'пейзаже', 'пейзажу', 'пейзажи'], '', 'за')

'пейзажи'

## Алгоритм спелл-чек

1. Поиск ошибок в тексте
2. Генерация кандидатов
3. Убийство кандидатов: если список слов оказывается пустым после проверки по словарю, возвращаем искомое слово (на случай, если его вдруг не оказалось в нашем словаре)
4. Считаем вероятность кандидата в контексте. Вставляем в текст кандидата с наибольшей вероятностью.
5. Возвращаем новый исправленный текст.

In [187]:
def spell_check(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    
    #поиск ошибок
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    for i, trigrams_string in enumerate(trigram_list):
        word = text.split()[i]
        for trigram in trigrams_string.split():
            if not(trigram_wiki_model.get(trigram)) or (trigram_wiki_model.get(trigram) < (10/8297222)):
                if word not in rucorpora_dict:
                    #print(word)
                    
                    #генерируем кандидатов + убираем невозможные по 3-грамной модели, оставшиеся проверяем по словарю
                    candidates = kill_candidates(word)
                    #print(candidates)
                    if (candidates): #если множество оставшихся кандидатов не пустое
                        if(i > 0): #если слово не первое в тексте, находим левый контекст
                            left_context = text.split()[i-1]
                        if i + 1 < len(text.split()): # если слово не последнее в тексте
                            right_context = text.split()[i+1]
                        #определяем наиболее вероятного
                        word = you_re_simply_the_best(candidates, left_context, right_context)
                    
        text_correct += ' ' + word
    return text_correct.strip()
                

In [183]:
train_df.wrong[9]

'пейзажы за окном какие то незнакомые'

In [186]:
%%time
spell_check(train_df.wrong[9])

пейзажы
['пейзаж', 'пейзажа', 'пейзажны', 'пейзаны', 'пейзаже', 'пейзажу', 'пейзажи']
Wall time: 1min 16s


'пейзажи за окном какие то незнакомые'

## Ура! Хотя бы на одном предложении сработало